# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

### Görev 1: Veriyi Hazırlama ve Analiz Etme

In [2]:
# Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [6]:
control_group = pd.read_excel('ab_testing.xlsx' , sheet_name="Control Group")
test_group = pd.read_excel('ab_testing.xlsx' , sheet_name="Test Group")

In [8]:
# Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

def check_dataframe(dataframe, head=5):
    print("Shape")
    print(dataframe.shape)
    print("Types")
    print(dataframe.dtypes)
    print("Head")
    print(dataframe.head())
    print("Tail")
    print(dataframe.tail())
    print("NA")
    print(dataframe.isnull().sum())
    print("Quantiles")
    print(dataframe.quantile([0, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99, 1]).T)

check_dataframe(control_group)

Shape
(40, 4)
Types
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
Head
    Impression      Click  Purchase    Earning
0  82529.45927 6090.07732 665.21125 2311.27714
1  98050.45193 3382.86179 315.08489 1742.80686
2  82696.02355 4167.96575 458.08374 1797.82745
3 109914.40040 4910.88224 487.09077 1696.22918
4 108457.76263 5987.65581 441.03405 1543.72018
Tail
     Impression      Click  Purchase    Earning
35 132064.21900 3747.15754 551.07241 2256.97559
36  86409.94180 4608.25621 345.04603 1781.35769
37 123678.93423 3649.07379 476.16813 2187.72122
38 101997.49410 4736.35337 474.61354 2254.56383
39 121085.88122 4285.17861 590.40602 1289.30895
NA
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
Quantiles
               0.00000     0.05000     0.25000     0.50000      0.75000      0.95000      0.99000      1.00000
Impression 45475.94296 79412.01792 85726.69035 99790.70108 115212.81654 132950.53245 143105.79110

In [9]:
check_dataframe(test_group)

Shape
(40, 4)
Types
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
Head
    Impression      Click  Purchase    Earning
0 120103.50380 3216.54796 702.16035 1939.61124
1 134775.94336 3635.08242 834.05429 2929.40582
2 107806.62079 3057.14356 422.93426 2526.24488
3 116445.27553 4650.47391 429.03353 2281.42857
4 145082.51684 5201.38772 749.86044 2781.69752
Tail
     Impression      Click  Purchase    Earning
35  79234.91193 6002.21358 382.04712 2277.86398
36 130702.23941 3626.32007 449.82459 2530.84133
37 116481.87337 4702.78247 472.45373 2597.91763
38  79033.83492 4495.42818 425.35910 2595.85788
39 102257.45409 4800.06832 521.31073 2967.51839
NA
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
Quantiles
               0.00000     0.05000      0.25000      0.50000      0.75000      0.95000      0.99000      1.00000
Impression 79033.83492 83150.50378 112691.97077 119291.30077 132050.57893 153178.69106 158245.2

In [12]:
# Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

control_group["group"] = "control"
test_group["group"] = "test"
combined_data = pd.concat([control_group,test_group], axis=0)
combined_data.head()

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
1,98050.45193,3382.86179,315.08489,1742.80686,control
2,82696.02355,4167.96575,458.08374,1797.82745,control
3,109914.40040,4910.88224,487.09077,1696.22918,control
4,108457.76263,5987.65581,441.03405,1543.72018,control


In [13]:
combined_data.shape

(80, 5)

### Görev 2: A/B Testinin Hipotezinin Tanımlanması

In [15]:
# Adım 1: Hipotezi tanımlayınız.
# H0 : M1 = M2
# H1 : M1!= M2

### H0 : M1 = M2 -> Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur
### H1 : M1!= M2 -> Kontrol grubu ve test grubu satın alma ortalamaları arasında fark vardır

In [16]:
# Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.
combined_data.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89406
test,582.10610


### Görev 3: Hipotez Testinin Gerçekleştirilmesi

In [20]:
#Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
#Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz.
# normallik varsayımı:
test_stat, pvalue = shapiro(combined_data.loc[combined_data["group"] == "test", "Purchase"])
print('Test Stat for Test Group = %.4f, p-value = %.4f' % (test_stat, pvalue))
test_stat, pvalue = shapiro(combined_data.loc[combined_data["group"] == "control", "Purchase"])
print('Test Stat for Control Group = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat for Test Group = 0.9589, p-value = 0.1541
Test Stat for Control Group = 0.9773, p-value = 0.5891


- eğer
p < 0.05 H0 reddedilir
p > 0.05 H0 reddedilemez

burada control grubu için p > 0.05 olduğu için HO reddedilemez
control grubu normal dağılım varsayımını sağlar

In [21]:
# varyans homojenliği:
test_stat, pvalue = levene(combined_data.loc[combined_data["group"] == "control", "Purchase"],
                           combined_data.loc[combined_data["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


- eğer
p < 0.05 H0 reddedilir
p > 0.05 H0 reddedilemez

burada p > 0.05 olduğu için H0 reddedilemez
control grubu ve test grubu varyans homejenliği varsayımını sağlar yani varyanslar homojendir diyebiliriz.

In [ ]:
# Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

# testler sonucu varsayımlar sağlandığı için bağımsız iki örneklem  t-testi uygulamalıyız.
# H0: M1 = M2 -> Kontrol grubu ve test grubu satın alma ortalamaları arasında anlamlı bir fark yoktur
# H1: M1 != M2 -> Kontrol grubu ve test grubu satın alma ortalamaları arasında anlamlı bir fark vardır

In [22]:
# Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki
# olarak anlamlı bir fark olup olmadığını yorumlayınız.

test_stat, pvalue = ttest_ind(combined_data.loc[combined_data["group"] == "control", "Purchase"],
                              combined_data.loc[combined_data["group"] == "test", "Purchase"],
                              equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


- eğer
p < 0.05 H0 reddedilir
p > 0.05 H0 reddedilemez

p-value=0.3493 bu yüzden HO reddedilemez.kontrol ve test grubu satın alma ortalamaları arasında anlamlı bir fark yoktur diyebiliriz

### Görev 4: Sonuçların Analizi

Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.
- önce hipotez testimizi gerçekleştirdik. hangi testi yapacağımızı daha iyi anlamak için varsayım kontrolleri yaptık. normallik varsayımına da varyans homojenliği varsayımına da uyduğu için iki örneklem t testini gerçekleştirdik

Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.
-

Elde ettiğimiz test sonuçlarına göre kontrol ve test grubu arasında anlamlı bir fark bulunmamaktadır, bu yüzden uygulanan yeni yönteminin çok etkili olmadığını söyleyebiliriz. Bunun yerine mevcut stratejilere devam edilmesi daha mantıklıdır.